In [ ]:
from __init__ import *

In [ ]:
w_df_, h_df_ = read_factorization(
    "../output/matrix_factorization/", SETTING["model_mode"], SETTING["model_data_"]
)

In [ ]:
distance_function = kraft.information.get_icd

In [ ]:
size = 800

layout = {"height": size, "width": size}

In [ ]:
factor_label_ = h_df_[0].index.to_numpy()

In [ ]:
w_node_x_node_distance_matrix_ = []

for index, df in enumerate(w_df_):

    matrix = df.to_numpy().T

    node_x_node_distance = kraft.array.function_on_2_2d_array(
        matrix, matrix, distance_function
    )

    kraft.plot.plot_heat_map(
        node_x_node_distance,
        factor_label_,
        factor_label_,
        "Factor",
        "Factor",
        layout={"title": {"text": "W {} Distance".format(index)}, **layout},
    )

    w_node_x_node_distance_matrix_.append(node_x_node_distance)

In [ ]:
h_node_x_node_distance_matrix_ = []

for index, df in enumerate(h_df_):

    matrix = df.to_numpy()

    node_x_node_distance = kraft.array.function_on_2_2d_array(
        matrix, matrix, distance_function,
    )

    kraft.plot.plot_heat_map(
        node_x_node_distance,
        factor_label_,
        factor_label_,
        "Factor",
        "Factor",
        layout={"title": {"text": "H {} Distance".format(index)}, **layout},
    )

    h_node_x_node_distance_matrix_.append(node_x_node_distance)

In [ ]:
node_x_node_distance_matrix = np.median(
    np.asarray(w_node_x_node_distance_matrix_ + h_node_x_node_distance_matrix_), axis=0
)

kraft.plot.plot_heat_map(
    node_x_node_distance_matrix,
    factor_label_,
    factor_label_,
    "Factor",
    "Factor",
    layout={"title": {"text": "W H Distance"}, **layout},
)

In [ ]:
node_marker_size = 24

if 1 < len(SETTING["resolution_"]):

    r_ = np.asarray(
        tuple(int(label.split(maxsplit=1)[1][0]) for label in factor_label_)
    )

    node_marker_size *= 2 ** (1 - kraft.array.normalize(r_, "0-1"))

In [ ]:
directory_path = "../output/gps_map/"

kraft.path.make(directory_path)

In [ ]:
for w_h_symbol in ("w", "h"):

    if w_h_symbol == "w":

        point_x_node = pd.concat(w_df_)

    else:

        point_x_node = pd.concat(h_df_, axis=1).T

    point_x_node **= SETTING["{}_gps_map_node_power".format(w_h_symbol)]

    # TODO: make a gps map know its node marke size
    gps_map = kraft.gps_map.GPSMap(node_x_node_distance, point_x_node)

    kraft.gps_map.write(
        "{}{}_gps_map.pickle.gz".format(directory_path, w_h_symbol), gps_map
    )

    gps_map.plot(
        node_trace={"marker": {"size": node_marker_size}},
        point_trace={
            "marker": {"size": SETTING["{}_gps_map_point_size".format(w_h_symbol)]}
        },
        file_path="{}{}_gps_map.html".format(directory_path, w_h_symbol),
    )